In [1]:
import pandas as pd
df = pd.read_json('result_data.json')

In [2]:
from datetime import datetime 

def change_time(dt):
    return datetime.fromtimestamp(dt//1000)

df['fetchdate_orig'] = df['fetchdate_orig'].apply(change_time)
df['fetchdate_check'] = df['fetchdate_check'].apply(change_time)

In [3]:
df_copy = df

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5040743 entries, 0 to 5040742
Data columns (total 9 columns):
 #   Column            Dtype         
---  ------            -----         
 0   fetchdate_orig    datetime64[ns]
 1   fetchdate_check   datetime64[ns]
 2   status            object        
 3   page_title        object        
 4   page_domain_root  object        
 5   page_url          object        
 6   changes           object        
 7   type              object        
 8   country           object        
dtypes: datetime64[ns](2), object(7)
memory usage: 384.6+ MB


In [1]:
import numpy as np
from collections import Counter
import pymorphy2
import pandas as pd
import json
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import time


In [2]:
morph = pymorphy2.MorphAnalyzer()

"""Function to convert words to initial form"""
def normal_rus(w):
    global morph
    return morph.parse(w)[0].normal_form

In [13]:
"""Create list of stopwords"""
stopwords = ["и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его", 
"но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от", "меня", "еще", 
"нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже", "или", "ни", "быть", "был", 
"него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они", 
"тут", "где", "есть", "надо", "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", 
"раз", "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем", 
"ним", "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех", 
"никогда", "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через", "эти", 
"нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем", "хорошо", "свою", "этой", 
"перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более", "всегда", "конечно", "всю", "между", 'иза']

date = datetime.strptime("01-01-2022", "%d-%m-%Y") # start day to analyse and vizualise wordcloud
csv = 'date, word, count\n' # structure for csv-file


"""Iterate over each day for a weekly period (total 120 days)"""

with open('analysis_count_words.csv', 'w') as file:
    for day in tqdm(range(120), desc = 'Loading'):
        cloud_week = df[(df['fetchdate_check'] >= date) & (df['fetchdate_check'] < date + timedelta(days=1))]
        text = " ".join(review for review in cloud_week.page_title)
        texts_lower_case = text.lower()
        only_words_text = re.sub('[^А-яЁёA-z]', ' ', texts_lower_case)
        texts_splitted = only_words_text.split()
        without_sw = [text for text in texts_splitted if text not in stopwords]
        c_dict = Counter()
        for w in without_sw:
            n = normal_rus(w)
            c_dict[n] += 1
        # generate and save wordclouds
        date_name = date.strftime('%d-%m-%Y')
        common = c_dict.most_common(100)
        if len(common) != 0:
            # save file
            for value in common:
                csv += (date_name + ', ' + value[0] + ', ' + str(value[1]) + '\n')
        date += timedelta(days=1)
        
    file.write(csv)

Loading: 100%|██████████| 120/120 [4:46:06<00:00, 143.06s/it]
